In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D, MaxPooling1D, Conv1D, Bidirectional
from tensorflow.keras.layers import Embedding
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_csv('/kaggle/input/zomato-reviews-ratings/zomato_reviews.csv')
df.head()

,Unnamed: 0,rating,review
0,0,5,nice
1,1,5,"best biryani , so supportive staff of outlet ,..."
2,2,4,delivery boy was very decent and supportive.👌👍
3,3,1,"worst biryani i have tasted in my life, half o..."
4,4,5,all food is good and tasty . will order again ...


In [3]:
print("Number of rows in data:", df.shape[0])
print("Number of columns in data", df.shape[1])

Number of rows in data: 5479
Number of columns in data 3


In [4]:
df.rating.value_counts()

rating
5    2288
1    1891
3     474
4     458
2     368
Name: count, dtype: int64

In [5]:
df.isnull().sum()

Unnamed: 0    0
rating        0
review        1
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.isnull().sum()

Unnamed: 0    0
rating        0
review        0
dtype: int64

In [8]:
df = df.drop(columns=['Unnamed: 0'])

In [9]:
df.rating = df.rating.replace([1,2,3],0)
df.rating = df.rating.replace([4,5],1)

In [10]:
df.head()

,rating,review
0,1,nice
1,1,"best biryani , so supportive staff of outlet ,..."
2,1,delivery boy was very decent and supportive.👌👍
3,0,"worst biryani i have tasted in my life, half o..."
4,1,all food is good and tasty . will order again ...


In [11]:
df.rating.value_counts()

rating
1    2745
0    2733
Name: count, dtype: int64

In [12]:
X = df["review"]
y = df["rating"]

In [13]:
X.head()

0                                                 nice
1    best biryani , so supportive staff of outlet ,...
2       delivery boy was very decent and supportive.👌👍
3    worst biryani i have tasted in my life, half o...
4    all food is good and tasty . will order again ...
Name: review, dtype: object

In [14]:
def stringprocess(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    
    return text

In [29]:
!pip install nltk

In [28]:
import nltk
import subprocess
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
from string import digits

# Download the wordnet resource using NLTK downloader
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Unzip the downloaded wordnet data manually if needed
wordnet_zip_path = "/root/nltk_data/corpora/wordnet.zip"
wordnet_dir_path = "/root/nltk_data/corpora/wordnet"

# Check if the wordnet directory already exists
if not os.path.exists(wordnet_dir_path):
    if os.path.exists(wordnet_zip_path):
        subprocess.run(["unzip", wordnet_zip_path, "-d", "/root/nltk_data/corpora/"])
    else:
        print(f"Zip file not found at {wordnet_zip_path}")
else:
    print("WordNet is already unzipped and available.")
    
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Zip file not found at /root/nltk_data/corpora/wordnet.zip
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/w

In [29]:
from string import digits
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def  textpreprocess(text):
    
    text = map(lambda x: x.lower(), text) # Lower case
    text = map(lambda x: re.sub(r"https?://\S+|www\.\S+", "", x), text) # Remove Links
    text = map(lambda x: re.sub(re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"),"", x), text) # Remove html tags
    text = map(lambda x: re.sub(r'[^\x00-\x7f]',r' ', x), text) # Remove non-ASCII characters 
    # Remove special special characters, including symbols, emojis, and other graphic characters

    emoji_pattern = re.compile(
            '['
            u'\U0001F600-\U0001F64F'  # emoticons
            u'\U0001F300-\U0001F5FF'  # symbols & pictographs
            u'\U0001F680-\U0001F6FF'  # transport & map symbols
            u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
            u'\U00002702-\U000027B0'
            u'\U000024C2-\U0001F251'
            ']+',
            flags=re.UNICODE)

    text = map(lambda x: emoji_pattern.sub(r'', x), text) 
    text = map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), text) # Remove punctuations
    
#     text = text.apply(lambda x: TextBlob(x).correct()) # Spelling correction
    
    remove_digits = str.maketrans('', '', digits)
    text = [i.translate(remove_digits) for i in text]
    text = [w for w in text if not w in stop_words]
    text = ' '.join([lemmatizer.lemmatize(w) for w in text])
    text = text.strip()
    return text

In [30]:
X = X.apply(lambda x: stringprocess(x))
word_tokens = X.apply(lambda x: word_tokenize(x))

preprocess_text = word_tokens.apply(lambda x: textpreprocess(x))

In [32]:
training_portion = 0.9
train_size = int(len(preprocess_text) * training_portion)

train_data = preprocess_text[0: train_size]
train_labels = np.array(y[0: train_size])

validation_data = preprocess_text[train_size:]
validation_labels = np.array(y[train_size:])


print(len(train_data))
print(len(train_labels))
print(len(validation_data))
print(len(validation_labels))

4930
4930
548
548


In [33]:
vocab_size = 5000
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(preprocess_text)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'br': 2,
 'good': 3,
 'food': 4,
 'taste': 5,
 'order': 6,
 'bad': 7,
 'quality': 8,
 'pizza': 9,
 'nice': 10}

In [34]:
train_sequences = tokenizer.texts_to_sequences(train_data)
print(train_sequences[10])

[71, 411, 1212, 5, 18, 723, 313, 2233]


In [35]:
embedding_dim = 32
max_length = 70
trunc_type = 'post'  # remove or truncate last words in sentences if max_length > 50 ans "post" defined last at sentence
padding_type = 'post'

In [36]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[1]))
print(len(train_padded[1]))
print(train_padded.shape)

12
70
(4930, 70)


In [37]:
validation_sequences = tokenizer.texts_to_sequences(validation_data)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

548
(548, 70)


In [38]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_data(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_data(train_padded[10]))
print('---')
print(train_data[10])

full paisa vasool taste best bawarchi aur everest ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
---
full paisa vasool taste best bawarchi aur everest


In [39]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=256))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 20
history = model.fit(train_padded, train_labels, epochs=num_epochs, verbose=2, validation_data=(validation_padded, validation_labels))

Epoch 1/20
155/155 - 17s - 107ms/step - accuracy: 0.5016 - loss: 0.6937 - val_accuracy: 0.3723 - val_loss: 0.7012
Epoch 2/20
155/155 - 9s - 56ms/step - accuracy: 0.5037 - loss: 0.6933 - val_accuracy: 0.4672 - val_loss: 0.6918
Epoch 3/20
155/155 - 8s - 54ms/step - accuracy: 0.5487 - loss: 0.6873 - val_accuracy: 0.4051 - val_loss: 0.7052
Epoch 4/20
155/155 - 8s - 53ms/step - accuracy: 0.6095 - loss: 0.6569 - val_accuracy: 0.4453 - val_loss: 0.7833
Epoch 5/20
155/155 - 11s - 68ms/step - accuracy: 0.6671 - loss: 0.5997 - val_accuracy: 0.5456 - val_loss: 0.7895
Epoch 6/20
155/155 - 10s - 64ms/step - accuracy: 0.7233 - loss: 0.5254 - val_accuracy: 0.5109 - val_loss: 0.8606
Epoch 7/20
155/155 - 8s - 54ms/step - accuracy: 0.7576 - loss: 0.4696 - val_accuracy: 0.4635 - val_loss: 1.0867
Epoch 8/20
155/155 - 8s - 53ms/step - accuracy: 0.7830 - loss: 0.4229 - val_accuracy: 0.4690 - val_loss: 1.2220
Epoch 9/20
155/155 - 9s - 56ms/step - accuracy: 0.8008 - loss: 0.4040 - val_accuracy: 0.4964 - val_l

In [41]:
string = ['worst biryani i have tasted in my life and food quality was also bad']
#vectorizing the tweet by the pre-fitted tokenizer instance
token = tokenizer.texts_to_sequences(string)
#padding the tweet to have exactly the same shape as `embedding_2` input
token_list = pad_sequences(token, maxlen=max_length-1, padding=padding_type, truncating=trunc_type)
# print(token_list)
sentiment = model.predict(token_list,batch_size=2,verbose = 2)[0]
print(string)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

1/1 - 1s - 585ms/step
['worst biryani i have tasted in my life and food quality was also bad']
negative
